In [1]:
import glob
import os
import fitz
import re

In [220]:
%load_ext autoreload
%autoreload 2

In [2]:
glob.glob("datasets/*")

['datasets/Perguntas e Respostas ITR 2024.pdf',
 'datasets/P&R IRPF 2023 - v1.1 - 04042023.pdf',
 'datasets/2024 Perguntas e Respostas da Pessoa Jurídica v30-8-2024.pdf',
 'datasets/P&R IRPF 2024 - v1.0 - 2024.05.03.pdf']

In [3]:
qna_irpf = fitz.open('datasets/P&R IRPF 2024 - v1.0 - 2024.05.03.pdf')

In [4]:
qna_irpf.page_count

301

In [198]:
QUESTIONS_PROCESSING_PATTERNS={
    "NEW_QUESTION": "^([0-9]{3}[0-9]?)\s?[—–-]\s?(.+)",
    "MULTI_LINE_QUESTION":"^(.+\?)",
    "END_OF_QUESTION": "^Retorno ao sumário"
}

In [247]:
ANSWER_REFERENCES_PATTERN=".+\((.+)\)\.?$|.+\((.+)\)\.?\s*\n.*([Cc]onsulte.+pergunta.+)|.+([Cc]onsulte.+pergunta.+)$"

In [213]:
def process_answer_body(which_answer):
    m = re.match (ANSWER_REFERENCES_PATTERN, "\n".join(which_answer), flags=re.DOTALL)

    references = ""
    linked_questions = ""
    end_of_answer_offset = 0
    
    if m is not None:
        if m.group(1) is not None:
            references = m.group(1)
            end_of_answer_offset = m.group(1).count('\n') + 1
            
        elif m.group(2) is not None:
            references = m.group(2)
            
            linked_questions = re.findall("\d+", m.group(3))

            end_of_answer_offset = m.group(2).count('\n') + m.group(3).count('\n') + 2
        elif m.group(4) is not None:
            linked_questions = re.findall("\d+", m.group(4))
            
            end_of_answer_offset = m.group(4).count('\n') + 2
            
    return {"answer_cleaned": which_answer[:-end_of_answer_offset] if end_of_answer_offset > 0 else which_answer,
            "references": references,
            "linked_questions": linked_questions}

In [201]:
def process_single_page(page_lines, 
                        current_question,
                        state,
                        processed_questions):

    for line in page_lines[2:]:
    
        m = re.match(QUESTIONS_PROCESSING_PATTERNS[state['current_pattern']], line)
    
        if m is not None:
            if state['current_pattern'] == "NEW_QUESTION":
                if len(m.groups()) > 0:
                    current_question['question_number'] = m.group(1)
                    current_question['question_summary'] = state['current_last_line']
                    current_question['question_text'] = m.group(2).strip()
                    current_question['answer'] = []

                    print("\n")
                    print(current_question)
                    print("\n")
                    
                    if current_question['question_text'][-1] != "?":
                        state['current_pattern'] = "MULTI_LINE_QUESTION"
                    else:
                        state['current_pattern'] = "END_OF_QUESTION"

                    print(f"Começo pergunta. questão={current_question['question_number']}")
            
            elif state['current_pattern'] == "MULTI_LINE_QUESTION":
                if len(m.groups()) > 0:
                    current_question['question_text'] += " " + m.group(1)
    
                    state['current_pattern'] = "END_OF_QUESTION"

                    print(f"Achou fim pergunta. questão={current_question['question_number']}")
                else:
                    current_question['question_text'] += " " + line
    
            elif state['current_pattern'] == "END_OF_QUESTION": 

                processed_answer = process_answer_body(current_question['answer'])

                current_question['answer_cleaned'] = processed_answer['answer_cleaned']
                current_question['references'] = processed_answer['references']
                current_question['linked_questions'] = processed_answer['linked_questions']
                
                processed_questions.append(current_question)

                print(f"Achou fim. questão={current_question['question_number']}. Total={len(processed_questions)}")

                current_question = {}
                state['current_pattern'] = "NEW_QUESTION"
        
            else:
                raise ValueError(f"Invalid pattern {state['current_pattern']}")
        else:
            if len(line.strip()) > 0:
                state['current_last_line'] = line.strip()
        
                if state['current_pattern'] == "END_OF_QUESTION":
                    current_question['answer'].append(line.strip())

    return current_question, processing_state

In [202]:
def print_questions(questions_list):
    for which_question in questions_list:
        print("\n-----------------------------------------------\n")
        print(f"Question number: {which_question['question_number']}")
        print(f"Question summary: {which_question['question_summary']}")
        print(f"Question text: {which_question['question_text']}\n")
        
        whole_answer = "\n".join(which_question['answer'])
        answer_cleaned = "\n".join(which_question['answer_cleaned'])
    
        print(f"Answer:\n{whole_answer}\n")
        print(f"Answer cleaned:\n{answer_cleaned}\n")
        print(f"References:\n{which_question['references']}\n")
        print(f"Linked questions:\n{which_question['linked_questions']}\n")

## Extract the questions

In [250]:
questions = []
current_question = {}
processing_state={"current_pattern": "NEW_QUESTION",
                  "current_last_line": ""}

In [251]:
for which_page in range(21, 300):
    current_question, processing_state = process_single_page(qna_irpf.load_page(which_page).get_text("text").split("\n"),
                                                             current_question,
                                                             processing_state,
                                                             questions)



{'question_number': '001', 'question_summary': 'OBRIGATORIEDADE', 'question_text': 'Quem está obrigado a apresentar a Declaração de Ajuste Anual relativa ao exercício de 2024,', 'answer': []}


Começo pergunta. questão=001
Achou fim pergunta. questão=001
Achou fim. questão=001. Total=1


{'question_number': '002', 'question_summary': 'PESSOA FÍSICA DESOBRIGADA', 'question_text': 'Pessoa física desobrigada pode apresentar a Declaração de Ajuste Anual (DAA)?', 'answer': []}


Começo pergunta. questão=002
Achou fim. questão=002. Total=2


{'question_number': '003', 'question_summary': 'TITULAR OU SÓCIO DE EMPRESA', 'question_text': 'Contribuinte que é titular ou sócio de empresa está obrigado a apresentar a Declaração de', 'answer': []}


Começo pergunta. questão=003
Achou fim pergunta. questão=003
Achou fim. questão=003. Total=3


{'question_number': '004', 'question_summary': 'QUADRO SOCIETÁRIO OU ASSOCIADO DE COOPERATIVA', 'question_text': 'Contribuinte, que participou de quadro soci

In [252]:
print(len(questions))

715


In [207]:
print_questions(questions)


-----------------------------------------------

Question number: 001
Question summary: OBRIGATORIEDADE
Question text: Quem está obrigado a apresentar a Declaração de Ajuste Anual relativa ao exercício de 2024, ano-calendário de 2023?

Answer:
Está obrigada a apresentar a Declaração de Ajuste Anual (DAA) referente ao exercício de 2024, a pessoa
física residente no Brasil que, no ano-calendário de 2023:
1 - recebeu rendimentos tributáveis, sujeitos ao ajuste na declaração, cuja soma foi superior a R$ 30.639,90
(trinta mil, seiscentos e trinta e nove reais e noventa centavos);
2 - recebeu rendimentos isentos, não tributáveis ou tributados exclusivamente na fonte, cuja soma foi superior
a R$ 200.000,00 (duzentos mil reais);
3 - obteve, em qualquer mês, ganho de capital na alienação de bens ou direitos sujeito à incidência do imposto;
4 - realizou operações de alienação em bolsas de valores, de mercadorias, de futuros e assemelhadas:
a) cuja soma foi superior a R$ 40.000,00 (quarenta mil 

In [219]:
questions[669]

{'question_number': '670',
 'question_summary': 'FALECIMENTO DE INCORPORADOR',
 'question_text': 'Na hipótese de falecimento de pessoa física equiparada a pessoa jurídica pela promoção de cujus, em relação aos créditos vincendos e aos remanescentes?',
 'answer': ['A morte, enquanto termo final da personalidade, implica exclusão do de cujus do mundo jurídico, mas não a',
  'extinção dos efeitos tributários que decorrem do empreendimento imobiliário e alcançam o espólio, o cônjuge',
  'meeiro e os sucessores causa mortis.',
  'Assim, se a lei fiscal equiparou a pessoa física responsável pelo empreendimento imobiliário a pessoa jurídica,',
  'a equiparação se prolonga até os sucessores causa mortis, porque, em face das leis que disciplinam o',
  'parcelamento do solo, eles continuam loteadores. É o que se depreende do art. 29 da Lei nº 6.766, de 19 de',
  'dezembro de 1979: "Aquele que adquirir a propriedade loteada (...) por sucessão causa mortis sucederá o',
  'transmitente em todos os 

## Check for missing or wrongly numbered questions

In [206]:
last_question = 0

for which_question in questions:
    print(which_question['question_number'])

    if last_question + 1 != int(which_question['question_number']):
        print(f"Missing questions: last_question={last_question}, current_question={int(which_question['question_number'])}")

    last_question = int(which_question['question_number'])

001
002
003
004
005
006
007
008
009
010
011
012
013
014
015
016
017
018
019
020
021
022
023
024
025
026
027
028
029
030
031
032
033
034
035
036
037
038
039
040
041
042
043
044
045
046
047
048
049
050
051
052
053
054
055
056
057
058
059
060
061
062
063
064
065
066
067
068
069
070
071
072
073
074
075
076
077
078
079
080
081
0820
Missing questions: last_question=81, current_question=820
083
Missing questions: last_question=820, current_question=83
084
085
086
087
088
089
090
091
092
093
094
095
096
097
098
099
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221

## Format the legal references using LLM

In [259]:
import json

from llm_access import *

### API keys

Expect to have a json file containing all the API keys needed. Groq key should be under "groq" identifier.

In [221]:
API_KEYS_FILE="/work/api_keys_20240427.json"

In [224]:
groq_key = json.load(open(API_KEYS_FILE))['groq']

### Groq model to use

In [226]:
groq_llama32_90b_interface = groq_access(groq_key, GROQ_LLAMA3_2_90B_MODEL)

In [260]:
result = legal_references_formatting(groq_llama32_90b_interface, questions[91]['references'].replace("\n", " "))


Lei nº 10.406, de 10 de janeiro de 2002 - Código Civil, arts. 215 e 216; Lei nº 7.713, de 22 de dezembro de 1988, art. 34, parágrafo único; Regulamento do Imposto sobre a Renda - RIR/2018, art. 945, parágrafo único, aprovado pelo Decreto nº 9.580, de 22 de novembro de 2018; Instrução Normativa SRF nº 81, de 11 de outubro de 2001, arts. 17 a 19; e Instrução Normativa RFB nº 2.055, de 6 de dezembro de 2021, art. 14
[{'role': 'system', 'content': 'Leia a lista de referências jurídicas e processe as informações, separando-as de maneira estruturada. Produza uma resposta apenas com o JSON no formato a seguir, sem incluir comentários ou mensagens de erro adicionais: {"referências":[{"título": <nome-completo-da-lei-ou-documento-jurídico-incluindo-instrumento-aprovação>, "artigos": [{"artigo": <número-do-artigo>, "incisos": [<número-romano-inciso-1>, ..., <número-romano-inciso-n>], "parágrafos": ["único" | <número-parágrafo-1>, ..., <número-parágrafo-n>]}, ...], "anexos": ["único" | <número-ro

In [261]:
result['referências']

[{'título': 'Lei nº 10.406, de 10 de janeiro de 2002 - Código Civil',
  'artigos': [{'artigo': '215', 'incisos': [], 'parágrafos': []},
   {'artigo': '216', 'incisos': [], 'parágrafos': []}],
  'anexos': []},
 {'título': 'Lei nº 7.713, de 22 de dezembro de 1988',
  'artigos': [{'artigo': '34', 'incisos': [], 'parágrafos': ['único']}],
  'anexos': []},
 {'título': 'Regulamento do Imposto sobre a Renda - RIR/2018, aprovado pelo Decreto nº 9.580, de 22 de novembro de 2018',
  'artigos': [{'artigo': '945', 'incisos': [], 'parágrafos': ['único']}],
  'anexos': []},
 {'título': 'Instrução Normativa SRF nº 81, de 11 de outubro de 2001',
  'artigos': [{'artigo': '17', 'incisos': [], 'parágrafos': []},
   {'artigo': '18', 'incisos': [], 'parágrafos': []},
   {'artigo': '19', 'incisos': [], 'parágrafos': []}],
  'anexos': []},
 {'título': 'Instrução Normativa RFB nº 2.055, de 6 de dezembro de 2021',
  'artigos': [{'artigo': '14', 'incisos': [], 'parágrafos': []}],
  'anexos': []}]

In [256]:
questions[91]['references'].replace("\n", " ")

'Lei nº 10.406, de 10 de janeiro de 2002 - Código Civil, arts. 215 e 216; Lei nº 7.713, de 22 de dezembro de 1988, art. 34, parágrafo único; Regulamento do Imposto sobre a Renda - RIR/2018, art. 945, parágrafo único, aprovado pelo Decreto nº 9.580, de 22 de novembro de 2018; Instrução Normativa SRF nº 81, de 11 de outubro de 2001, arts. 17 a 19; e Instrução Normativa RFB nº 2.055, de 6 de dezembro de 2021, art. 14'

In [253]:
questions[91]

{'question_number': '092',
 'question_summary': 'PESSOA FALECIDA - RESTITUIÇÃO DO IMPOSTO SOBRE A RENDA',
 'question_text': 'É dispensável o alvará judicial na restituição, ao cônjuge viúvo ou aos herdeiros do falecido, inventário?',
 'answer': ['Existindo bens sujeitos a inventário ou a arrolamento, e tendo sido encerrado o inventário sem a inclusão do',
  'imposto sobre a renda não recebido em vida pelo titular, a restituição depende:',
  'a) de alvará judicial, caso o inventário tenha sido feito por processo judicial de inventário; ou',
  'b) de escritura pública de inventário e partilha, na hipótese de o inventário ter sido feito dessa forma.',
  'Não havendo bens sujeitos a inventário e existindo dependentes habilitados na forma da legislação',
  'previdenciária ou militar, a restituição é liberada mediante requerimento dirigido ao delegado da Delegacia da',
  'Receita Federal do Brasil da jurisdição do último endereço do de cujus.',
  'O requerimento deve ser formulado pelo cônju

In [21]:
" ".join(questions[1]['answer'])

'Sim. A pessoa física, ainda que desobrigada, pode apresentar a Declaração de Ajuste Anual (DAA), sendo vedado a um mesmo contribuinte constar simultaneamente em mais de uma Declaração de Ajuste Anual, seja como titular ou dependente, exceto nos casos de alteração na relação de dependência no ano-calendário de 2023. (Instrução Normativa RFB nº 2.178, de 5 de março de 2024, art. 2º, § 2º) Consulte as perguntas 001 e 174'

In [166]:
"\n".join(questions[33]['answer'])

'1 - Acesse o site da Secretaria Especial da Receita Federal do Brasil (RFB) na Internet, no endereço\nhttp://www.gov.br/receitafederal/pt-br;\n2 - Localize o programa IRPF 2024 na opção “Meu Imposto de Renda”;\n3 – Siga as orientações para download constantes no site da RFB na Internet.\nConsulte as perguntas 032, 035 e 036'

In [171]:
m = re.match (".+\((.+)\)$|.+\((.+)\)\s*\n.*([Cc]onsulte.+pergunta.+)$|.+([Cc]onsulte.+pergunta.+)$", "\n".join(questions[33]['answer']), flags=re.DOTALL)

In [172]:
m

<re.Match object; span=(0, 321), match='1 - Acesse o site da Secretaria Especial da Recei>

In [175]:
m.group(4)

'Consulte as perguntas 032, 035 e 036'

In [129]:
len(m.groups())

4

In [68]:
m.group(2).count('\n')

0

In [69]:
m.group(3).count('\n')

0

In [73]:
m.group(3)

'Consulte as perguntas 001 e 174'

In [90]:
questions[1]['answer'][:-2]

['Sim. A pessoa física, ainda que desobrigada, pode apresentar a Declaração de Ajuste Anual (DAA), sendo',
 'vedado a um mesmo contribuinte constar simultaneamente em mais de uma Declaração de Ajuste Anual, seja',
 'como titular ou dependente, exceto nos casos de alteração na relação de dependência no ano-calendário de',
 '2023.']

In [59]:
questions[0]['answer'][-4]

'(Lei nº 9.250, de 26 de dezembro de 1995, art. 25; Lei nº 9.779, de 19 de janeiro de 1999, art. 16;'

In [85]:
m2 = re.findall("\d+", m.group(3))

In [87]:
m2

['001', '174']